In [154]:
from numpy import mean
from numpy import std
from numpy import dstack
from pandas import read_csv
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import TimeDistributed
from keras.layers import ConvLSTM2D
from keras.utils import to_categorical
from matplotlib import pyplot

import numpy as np
import h5py
import pandas as pd

In [155]:
f = h5py.File('Z:\gait_database\LSTM_data_onlyMarkersEMG.h5', 'r')

In [156]:
print(list(f.keys()))

['x_test', 'x_train', 'x_valid', 'y_test', 'y_train', 'y_valid']


In [261]:
x_test = f['x_test'][:,:,-16:]
x_train = f['x_train'][:,:,-16:]
x_valid = f['x_valid'][:,:,-16:]
y_test = f['y_test']
y_train = f['y_train']
y_valid = f['y_valid']

In [262]:
print("print x_train =",x_train.shape, "y_train =",y_train.shape)
print("print x_test = ", x_test.shape,"y_test  =", y_test.shape)
print("print x_valid =",x_valid.shape, "y_valid =",y_valid.shape)

print x_train = (301, 425, 16) y_train = (301, 1)
print x_test =  (100, 425, 16) y_test  = (100, 1)
print x_valid = (100, 425, 16) y_valid = (100, 1)


In [263]:
import pandas as pd

In [264]:
def load_group(filenames):
    loaded = list()
    for name in filenames:
        data = f
        loaded.append(data)
    # grupę stosów, tak aby funkcje były trzecim wymiarem
    loaded = dstack(loaded)
    return loaded

In [265]:
    c = list(zip(x_train,y_train ))
    np.random.shuffle(c)
    d = list(zip(x_test,y_test ))
    np.random.shuffle(d)
    v = list(zip(c,d))
    np.random.shuffle(v)

In [270]:
def load_dataset():
    
    c = list(zip(x_train,y_train ))
    np.random.shuffle(c)
    d = list(zip(x_test,y_test ))
    np.random.shuffle(d)
    v = list(zip(c,d))
    np.random.shuffle(v)
    
    TrainX = x_train
    TestX = x_test
    TrainY = y_train
    TestY = y_test

    TrainY = to_categorical(TrainY)
    TestY = to_categorical(TestY)
    #print(trenujX.shape, trenujY.shape, testX.shape, testY.shape)
    return TrainX, TrainY, TestX, TestY

In [271]:
load_dataset()

(array([[[0.47765241, 0.38833949, 0.34111948, ..., 0.47068121,
          0.38283664, 0.3724125 ],
         [0.49767748, 0.39024425, 0.34914225, ..., 0.47443327,
          0.39793688, 0.38560514],
         [0.51771804, 0.39211173, 0.35709895, ..., 0.47809369,
          0.41272024, 0.39835801],
         ...,
         [0.58218069, 0.40065017, 0.39555756, ..., 0.48562331,
          0.41481817, 0.4630288 ],
         [0.58203488, 0.40064472, 0.39549365, ..., 0.48563665,
          0.41486074, 0.46317013],
         [0.5819889 , 0.40064312, 0.39547435, ..., 0.48564089,
          0.41487389, 0.46321681]],
 
        [[0.45392692, 0.38764917, 0.32675023, ..., 0.45113569,
          0.34808606, 0.38149922],
         [0.48644956, 0.39047375, 0.34743345, ..., 0.45359287,
          0.35720003, 0.38618735],
         [0.51752601, 0.39328   , 0.36701407, ..., 0.45542163,
          0.36595456, 0.39056048],
         ...,
         [0.549724  , 0.40928659, 0.48767483, ..., 0.552489  ,
          0.39007488, 0.

In [272]:
def evaluate_model(TrainX, TrainY, TestX, TestY):
    verbose, epochs, batch_size = 1, 10, 64 # 15, 64 domyslnie, dla 10, 64 działa najlepiej verbose pokaze liczbe epok
    
    n_timesteps, n_features, n_outputs = TrainX.shape[1], TrainX.shape[2], TrainY.shape[1]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.1)) # 0.5
    
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(TrainX, TrainY, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(TestX, TestY, batch_size=batch_size, verbose=1)
    return accuracy

In [273]:
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))

In [274]:
def run_experiment(repeats=10):
    # load data
    TrainX, TrainY, TestX, TestY = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(TrainX, TrainY, TestX, TestY)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)


In [275]:
run_experiment(repeats=10)

Epoch 1/10
5/5 [==============================] - 2s 361ms/step - loss: 0.6870 - accuracy: 0.4950
Epoch 2/10
5/5 [==============================] - 2s 370ms/step - loss: 0.6427 - accuracy: 0.7110
Epoch 3/10
5/5 [==============================] - 2s 374ms/step - loss: 0.5361 - accuracy: 0.8206
Epoch 4/10
5/5 [==============================] - 2s 371ms/step - loss: 0.4463 - accuracy: 0.8140
Epoch 5/10
5/5 [==============================] - 2s 363ms/step - loss: 0.3430 - accuracy: 0.8671
Epoch 6/10
5/5 [==============================] - 2s 380ms/step - loss: 0.2456 - accuracy: 0.9203
Epoch 7/10
5/5 [==============================] - 2s 374ms/step - loss: 0.2194 - accuracy: 0.9236
Epoch 8/10
5/5 [==============================] - 2s 383ms/step - loss: 0.2761 - accuracy: 0.8904
Epoch 9/10
5/5 [==============================] - 2s 374ms/step - loss: 0.2652 - accuracy: 0.9103
Epoch 10/10
2/2 [==============================] - 0s 47ms/step - loss: 1.4834 - accuracy: 0.5200
>#1: 52.000
Epoch 1/

5/5 [==============================] - 2s 385ms/step - loss: 0.1191 - accuracy: 0.9668
Epoch 10/10
2/2 [==============================] - 0s 45ms/step - loss: 2.0537 - accuracy: 0.5200
>#5: 52.000
Epoch 1/10
5/5 [==============================] - 2s 376ms/step - loss: 0.7026 - accuracy: 0.5249
Epoch 2/10
5/5 [==============================] - 2s 379ms/step - loss: 0.6776 - accuracy: 0.5681
Epoch 3/10
5/5 [==============================] - 2s 370ms/step - loss: 0.6344 - accuracy: 0.7608
Epoch 4/10
5/5 [==============================] - 2s 384ms/step - loss: 0.5303 - accuracy: 0.8372
Epoch 5/10
5/5 [==============================] - 2s 379ms/step - loss: 0.4434 - accuracy: 0.8140
Epoch 6/10
5/5 [==============================] - 2s 382ms/step - loss: 0.4210 - accuracy: 0.8605
Epoch 7/10
5/5 [==============================] - 2s 374ms/step - loss: 0.4519 - accuracy: 0.8405
Epoch 8/10
5/5 [==============================] - 2s 389ms/step - loss: 0.3645 - accuracy: 0.8704
Epoch 9/10
5/5 [===

2/2 [==============================] - 0s 51ms/step - loss: 0.9193 - accuracy: 0.6000
>#9: 60.000
Epoch 1/10
5/5 [==============================] - 2s 379ms/step - loss: 0.6853 - accuracy: 0.5648
Epoch 2/10
5/5 [==============================] - 2s 384ms/step - loss: 0.6435 - accuracy: 0.6312
Epoch 3/10
5/5 [==============================] - 2s 360ms/step - loss: 0.5680 - accuracy: 0.7475
Epoch 4/10
5/5 [==============================] - 2s 387ms/step - loss: 0.4709 - accuracy: 0.7641
Epoch 5/10
5/5 [==============================] - 2s 338ms/step - loss: 0.4080 - accuracy: 0.8239
Epoch 6/10
5/5 [==============================] - 2s 371ms/step - loss: 0.2479 - accuracy: 0.9169
Epoch 7/10
5/5 [==============================] - 2s 390ms/step - loss: 0.1631 - accuracy: 0.9535
Epoch 8/10
5/5 [==============================] - 2s 390ms/step - loss: 0.1098 - accuracy: 0.9801
Epoch 9/10
5/5 [==============================] - 2s 386ms/step - loss: 0.1067 - accuracy: 0.9701
Epoch 10/10
2/2 [===